# 기상청 기상데이터 정리 및 merge

## Contents
    1) 기상청 데이터 각각 전처리 (파일별 반복 생략)  
    2) 파일별 반복하며 전처리된 데이터 merge 수행

In [2]:
# 필요한 라이브러리 로딩 
import pandas as pd

# 확인 가능한 row수 늘리기
pd.set_option("display.max_rows", 100)

## 1) 기상청 데이터 각각 전처리
    - 데이터 다운로드 
        - url : https://data.kma.go.kr/cmmn/main.do (기상자료개방포털) 
        - 포탈 내 접근 : 데이터 > 기상예보 > 동네예보 
        - 동해시 북평동 2018.1~12 선택
        - 강수, 강수형태, 기온, 뇌전, 습도, 풍속, 풍향, 하늘상태 다운로드 
        - 한번에 다운시 파일명 에러발생하여, 각각 다운 후 별도 저장 후 작업 실시  
    
    - 작업내용 
        - 날짜(연-월), 시간, 생성
        - 시간 0,1,2,3,4,5, 21,22,23 제거, 06~20만 남기기 위함
        - 강수, 강수형태, 기온, 뇌전, 습도, 풍속, 풍향, 하늘상태 반복하여 실시함 (반복실시한 내용은 생략함)
       

In [33]:
# 데이터 불러오기
rain = pd.read_csv('동해시_북평동_하늘상태2018.csv')

# 반복시
# rain = pd.read_csv('동해시_북평동_풍속2018.csv')
# rain = pd.read_csv('동해시_북평동_풍향2018.csv')
# rain = pd.read_csv('동해시_북평동_뇌전2018.csv')
# rain = pd.read_csv('동해시_북평동_기온2018.csv')
# rain = pd.read_csv('동해시_북평동_강수2018.csv')
# rain = pd.read_csv('동해시_북평동_습도2018.csv')
# rain = pd.read_csv('동해시_북평동_강수형태2018.csv')

In [34]:
rain.head()

,format: day,hour,value location:97_126 Start : 20180101
0,1,0.0,1.0
1,1,100.0,1.0
2,1,200.0,1.0
3,1,300.0,1.0
4,1,400.0,1.0


In [35]:
# column명 변경
rain.columns = ['day', 'hour', 'sky_con']

In [36]:
# 중간중간 월 row가 끼어 있음
rain['day'].unique()

array([' 1', ' 2', ' 3', ' 4', ' 5', ' 6', ' 7', ' 8', ' 9', ' 10', ' 11',
       ' 12', ' 13', ' 14', ' 15', ' 16', ' 17', ' 18', ' 19', ' 20',
       ' 21', ' 22', ' 23', ' 24', ' 25', ' 26', ' 27', ' 28', ' 29',
       ' 30', ' 31', ' Start : 20180201 ', ' Start : 20180301 ',
       ' Start : 20180401 ', ' Start : 20180501 ', ' Start : 20180601 ',
       ' Start : 20180701 ', ' Start : 20180801 ', ' Start : 20180901 ',
       ' Start : 20181001 ', ' Start : 20181101 ', ' Start : 20181201 '],
      dtype=object)

In [37]:
# 월 column을 만들기 위해서 인덱스 추출 
feb_idx = rain[rain['day'] == ' Start : 20180201 '].index[0]
mar_idx = rain[rain['day'] == ' Start : 20180301 '].index[0]
apr_idx = rain[rain['day'] == ' Start : 20180401 '].index[0]
may_idx = rain[rain['day'] == ' Start : 20180501 '].index[0]
jun_idx = rain[rain['day'] == ' Start : 20180601 '].index[0]
jul_idx = rain[rain['day'] == ' Start : 20180701 '].index[0]
aug_idx = rain[rain['day'] == ' Start : 20180801 '].index[0]
sep_idx = rain[rain['day'] == ' Start : 20180901 '].index[0]
oct_idx = rain[rain['day'] == ' Start : 20181001 '].index[0]
nov_idx = rain[rain['day'] == ' Start : 20181101 '].index[0]
dec_idx = rain[rain['day'] == ' Start : 20181201 '].index[0]

In [38]:
# 빈 "month" column 생성
rain['month'] = 0

In [39]:
# 위에서 추출한 월 index를 가지고, iloc 인덱싱을 활용하여 1~12월 입력
rain.iloc[:feb_idx, 3] = 1
rain.iloc[feb_idx:mar_idx, 3] = 2
rain.iloc[mar_idx:apr_idx, 3] = 3
rain.iloc[apr_idx:may_idx, 3] = 4
rain.iloc[may_idx:jun_idx, 3] = 5
rain.iloc[jun_idx:jul_idx, 3] = 6
rain.iloc[jul_idx:aug_idx, 3] = 7
rain.iloc[aug_idx:sep_idx, 3] = 8
rain.iloc[sep_idx:oct_idx, 3] = 9
rain.iloc[oct_idx:nov_idx, 3] = 10
rain.iloc[nov_idx:dec_idx, 3] = 11
rain.iloc[dec_idx:, 3] = 12

In [40]:
# na 제거, index reset
rain.dropna(inplace=True)
rain.reset_index(drop=True, inplace=True)
rain.head()

,day,hour,sky_con,month
0,1,0.0,1.0,1
1,1,100.0,1.0,1
2,1,200.0,1.0,1
3,1,300.0,1.0,1
4,1,400.0,1.0,1


In [41]:
# 시간이 1시가 100, 2시가 200, 표기 되어 있어서 100으로 나누어 줌
rain['hour']=rain.hour/100

In [42]:
# hours = []
# for i in range(len(rain['day'])):
#     hour = int(rain['hour'][i])
#     hours.append(hour)
# rain['hour'] = hours

# float -> int 형식으로 바꿈, 위 코드는 astype이 생각이 안나서 씀
rain['hour'] = rain['hour'].astype(int)

In [43]:
rain.head()

,day,hour,sky_con,month
0,1,0,1.0,1
1,1,1,1.0,1
2,1,2,1.0,1
3,1,3,1.0,1
4,1,4,1.0,1


In [44]:
# 해 없는 시간 제거하기 
# 'hour' column 에서 0,1,2,3,4,5, 21,22,23이 있는 index를 추출


result = []
rej_time = [0,1,2,3,4,5, 21,22,23]
for j in rej_time:
    one_indice = []
    for i in range(len(rain[rain['hour'] ==j].index)):
        one = rain[rain['hour'] ==j].index[i]
        one_indice.append(one)
        
    result.extend(one_indice)    

rain.drop(result, inplace=True)
rain.reset_index(drop=True, inplace=True)
rain.tail()

,day,hour,sky_con,month
5470,31,16,-1.0,12
5471,31,17,-1.0,12
5472,31,18,-1.0,12
5473,31,19,-1.0,12
5474,31,20,-1.0,12


In [45]:
# 연도는 모두 2018년 이기 때문에 입력
rain['year'] = '2018'

In [46]:
rain.head()

,day,hour,sky_con,month,year
0,1,6,1.0,1,2018
1,1,7,1.0,1,2018
2,1,8,1.0,1,2018
3,1,9,2.0,1,2018
4,1,10,2.0,1,2018


In [47]:
# months=[]
# for i in range(len(rain['day'])):
#     month = str(rain['month'][i])
#     months.append(month)
    
# rain['month'] = months

# "month" column -> string type으로 변경  
rain['month'] = rain['month'].astype(str)
rain.head()

,day,hour,sky_con,month,year
0,1,6,1.0,1,2018
1,1,7,1.0,1,2018
2,1,8,1.0,1,2018
3,1,9,2.0,1,2018
4,1,10,2.0,1,2018


In [49]:
# "day" column에 공백이 하나씩 있음
# 공백 제거 함수 strip이용
days = []
for i in range(len(rain['day'])):
    day = rain['day'][i].strip()
    days.append(day)
    
rain['day'] = days

In [50]:
# 연 - 월 - 일 형식으로  "date" column 생성
rain['date'] = rain['year'] + '-' + rain['month'] + '-' + rain['day']

In [51]:
rain.head()

,day,hour,sky_con,month,year,date
0,1,6,1.0,1,2018,2018-1-1
1,1,7,1.0,1,2018,2018-1-1
2,1,8,1.0,1,2018,2018-1-1
3,1,9,2.0,1,2018,2018-1-1
4,1,10,2.0,1,2018,2018-1-1


In [52]:
# 필요한 column만 선택함
rain = rain[['date','hour','sky_con']]
rain.head()

,date,hour,sky_con
0,2018-1-1,6,1.0
1,2018-1-1,7,1.0
2,2018-1-1,8,1.0
3,2018-1-1,9,2.0
4,2018-1-1,10,2.0


In [220]:
# 정리한 데이터 csv로 저장 
# 파일 덮어쓰기 될까봐 주석처리함
# 단축키 : ctrl + / 

# rain.to_csv('donghae_skycon2018.csv')



# 반복시
# rain.to_csv('donghae_windangle2018.csv')
# rain.to_csv('donghae_windvelo2018.csv')
# rain.to_csv('donghae_humidity2018.csv')
# rain.to_csv('donghae_thunder2018.csv')
# rain.to_csv('donghae_temp2018.csv')
# rain.to_csv('donghae_raintype2018.csv')
# rain.to_csv('donghae_rain2018.csv')


## 2) 전처리 데이터 merge 수행

1)에서 정리한 데이터들을 불러와서 merge 함수를 이용하여 데이터 결합

- 작업 내용 
    - 각각 데이터 불러오기 , 키값 만들기  
        반복 : (습도, 강수, 강수형태, 기온, 뇌전, 풍향, 풍속, 하늘상태)
    - merge로 붙이기 

### 습도

In [54]:
# 데이터 불러오기 , index_col =0 옵션 Unnamed: 0 칼럼 안생기게 함 
humidity = pd.read_csv('donghae_humidity2018.csv', index_col=0)
humidity.tail()
# index_col=0 기억 안날때, drop
# humidity.drop('Unnamed: 0', axis=1, inplace=True)

,date,hour,humidity
5470,2018-12-31,16,26.0
5471,2018-12-31,17,18.0
5472,2018-12-31,18,24.0
5473,2018-12-31,19,22.0
5474,2018-12-31,20,23.0


In [55]:
# "date_time" column을 key-값으로 사용할 예정

# "hour" column string type으로 변환
humidity.hour= humidity.hour.astype(str)

# 연 - 월 - 일 - 시간 형식으로 date_time 저장
humidity['date_time'] = humidity['date'] + '-' + humidity['hour']
humidity.head()

,date,hour,humidity,date_time
0,2018-1-1,6,17.0,2018-1-1-6
1,2018-1-1,7,19.0,2018-1-1-7
2,2018-1-1,8,14.0,2018-1-1-8
3,2018-1-1,9,18.0,2018-1-1-9
4,2018-1-1,10,20.0,2018-1-1-10


### 강수

In [56]:
rain = pd.read_csv('donghae_rain2018.csv')
rain.drop('Unnamed: 0', axis=1, inplace=True)
rain.head()

,date,hour,rain
0,2018-1-1,6,0.0
1,2018-1-1,7,0.0
2,2018-1-1,8,0.0
3,2018-1-1,9,0.0
4,2018-1-1,10,0.0


In [57]:
rain.hour= rain.hour.astype(str)
rain['date_time'] = rain['date'] + '-' + rain['hour']
rain.head()

,date,hour,rain,date_time
0,2018-1-1,6,0.0,2018-1-1-6
1,2018-1-1,7,0.0,2018-1-1-7
2,2018-1-1,8,0.0,2018-1-1-8
3,2018-1-1,9,0.0,2018-1-1-9
4,2018-1-1,10,0.0,2018-1-1-10


### 강수형태

In [58]:
raintype = pd.read_csv('donghae_raintype2018.csv')
raintype.drop('Unnamed: 0', axis=1, inplace=True)
raintype.head()

,date,hour,rain_type
0,2018-1-1,6,0.0
1,2018-1-1,7,0.0
2,2018-1-1,8,0.0
3,2018-1-1,9,0.0
4,2018-1-1,10,0.0


In [59]:
raintype.hour= raintype.hour.astype(str)
raintype['date_time'] = raintype['date'] + '-' + raintype['hour']
raintype.head()

,date,hour,rain_type,date_time
0,2018-1-1,6,0.0,2018-1-1-6
1,2018-1-1,7,0.0,2018-1-1-7
2,2018-1-1,8,0.0,2018-1-1-8
3,2018-1-1,9,0.0,2018-1-1-9
4,2018-1-1,10,0.0,2018-1-1-10


### 기온

In [60]:
temp = pd.read_csv('donghae_temp2018.csv')
temp.drop('Unnamed: 0', axis=1, inplace=True)
temp.head()

,date,hour,temp
0,2018-1-1,6,4.7
1,2018-1-1,7,3.4
2,2018-1-1,8,2.7
3,2018-1-1,9,2.3
4,2018-1-1,10,2.1


In [61]:
temp.hour= temp.hour.astype(str)
temp['date_time'] = temp['date'] + '-' + temp['hour']
temp.head()

,date,hour,temp,date_time
0,2018-1-1,6,4.7,2018-1-1-6
1,2018-1-1,7,3.4,2018-1-1-7
2,2018-1-1,8,2.7,2018-1-1-8
3,2018-1-1,9,2.3,2018-1-1-9
4,2018-1-1,10,2.1,2018-1-1-10


### 뇌전

In [62]:
thunder = pd.read_csv('donghae_thunder2018.csv')
thunder.drop('Unnamed: 0', axis=1, inplace=True)
thunder.head()

,date,hour,thunder
0,2018-1-1,6,0.0
1,2018-1-1,7,0.0
2,2018-1-1,8,0.0
3,2018-1-1,9,0.0
4,2018-1-1,10,0.0


In [63]:
thunder.hour= thunder.hour.astype(str)
thunder['date_time'] =thunder['date'] + '-' + thunder['hour']
thunder.head()

,date,hour,thunder,date_time
0,2018-1-1,6,0.0,2018-1-1-6
1,2018-1-1,7,0.0,2018-1-1-7
2,2018-1-1,8,0.0,2018-1-1-8
3,2018-1-1,9,0.0,2018-1-1-9
4,2018-1-1,10,0.0,2018-1-1-10


### 풍향

In [64]:
wind_angle = pd.read_csv('donghae_windangle2018.csv')
wind_angle.drop('Unnamed: 0', axis=1, inplace=True)
wind_angle.head()

,date,hour,wind_angle
0,2018-1-1,6,289.0
1,2018-1-1,7,278.0
2,2018-1-1,8,268.0
3,2018-1-1,9,268.0
4,2018-1-1,10,250.0


In [65]:
wind_angle.hour= wind_angle.hour.astype(str)
wind_angle['date_time'] =wind_angle['date'] + '-' + wind_angle['hour']
wind_angle.head()

,date,hour,wind_angle,date_time
0,2018-1-1,6,289.0,2018-1-1-6
1,2018-1-1,7,278.0,2018-1-1-7
2,2018-1-1,8,268.0,2018-1-1-8
3,2018-1-1,9,268.0,2018-1-1-9
4,2018-1-1,10,250.0,2018-1-1-10


### 풍속

In [66]:
wind_velo = pd.read_csv('donghae_windvelo2018.csv')
wind_velo.drop('Unnamed: 0', axis=1, inplace=True)
wind_velo.head()

,date,hour,wind_velo
0,2018-1-1,6,2.8
1,2018-1-1,7,2.7
2,2018-1-1,8,2.3
3,2018-1-1,9,2.3
4,2018-1-1,10,3.0


In [67]:
wind_velo.hour= wind_velo.hour.astype(str)
wind_velo['date_time'] =wind_velo['date'] + '-' + wind_velo['hour']
wind_velo.head()

,date,hour,wind_velo,date_time
0,2018-1-1,6,2.8,2018-1-1-6
1,2018-1-1,7,2.7,2018-1-1-7
2,2018-1-1,8,2.3,2018-1-1-8
3,2018-1-1,9,2.3,2018-1-1-9
4,2018-1-1,10,3.0,2018-1-1-10


### 하늘상태
- 결측치가 -1로 표기 되어있음
- 동해시 전운량 데이터 변경 후 대치 
    - 데이터 다운로드 
        - url : https://data.kma.go.kr/cmmn/main.do (기상자료개방포털) 
        - 포탈 내 접근 : 데이터 > 기상관측 > 지상 
        - 동해, 전운량 선택, 기간 2018.1/1~ 12/31 선택 후 다운로드 
    - 전운량은 0 ~ 10으로 되어 있기 때문에 기존 하늘상태 데이터와 맞추기 위해서는 1~4 범위로 변경 필요함 
        - 전운량 데이터도 7개 비어 있음
        - 엑셀에서 7개 값 채움

In [68]:
sky_con = pd.read_csv('donghae_skycon2018.csv')
sky_con.drop('Unnamed: 0', axis=1, inplace=True)
sky_con.head()

,date,hour,sky_con
0,2018-1-1,6,1.0
1,2018-1-1,7,1.0
2,2018-1-1,8,1.0
3,2018-1-1,9,2.0
4,2018-1-1,10,2.0


In [69]:
sky_con.hour= sky_con.hour.astype(str)
sky_con['date_time'] =sky_con['date'] + '-' + sky_con['hour']
sky_con.head()

,date,hour,sky_con,date_time
0,2018-1-1,6,1.0,2018-1-1-6
1,2018-1-1,7,1.0,2018-1-1-7
2,2018-1-1,8,1.0,2018-1-1-8
3,2018-1-1,9,2.0,2018-1-1-9
4,2018-1-1,10,2.0,2018-1-1-10


In [70]:
sky_con.sky_con.unique()

array([ 1.,  2.,  3.,  4., -1.])

In [71]:
# 10월 15일부터 12월 31일까지 결측치
sky_con[sky_con['sky_con'] ==-1]

,date,hour,sky_con,date_time
4305,2018-10-15,6,-1.0,2018-10-15-6
4306,2018-10-15,7,-1.0,2018-10-15-7
4307,2018-10-15,8,-1.0,2018-10-15-8
4308,2018-10-15,9,-1.0,2018-10-15-9
4309,2018-10-15,10,-1.0,2018-10-15-10
...,...,...,...,...
5470,2018-12-31,16,-1.0,2018-12-31-16
5471,2018-12-31,17,-1.0,2018-12-31-17
5472,2018-12-31,18,-1.0,2018-12-31-18
5473,2018-12-31,19,-1.0,2018-12-31-19


In [101]:
# 결측치 확인
sky_con.iloc[4305:]

,date,hour,sky_con,date_time
4305,2018-10-15,6,-1.0,2018-10-15-6
4306,2018-10-15,7,-1.0,2018-10-15-7
4307,2018-10-15,8,-1.0,2018-10-15-8
4308,2018-10-15,9,-1.0,2018-10-15-9
4309,2018-10-15,10,-1.0,2018-10-15-10
...,...,...,...,...
5470,2018-12-31,16,-1.0,2018-12-31-16
5471,2018-12-31,17,-1.0,2018-12-31-17
5472,2018-12-31,18,-1.0,2018-12-31-18
5473,2018-12-31,19,-1.0,2018-12-31-19


In [496]:
# sky_con_na = sky_con.iloc[4305:]

## 전운량 데이터 정리
- 하늘 상태 결측치 채우기 위해서 불러옴 
- 7개 결측치 있음, 엑셀에서 양쪽값의 평균치로 대치함

In [72]:
# 전운량 데이터 불러오기
cloud = pd.read_csv('동해시전운량.csv', encoding='cp949')
cloud.head()

,지점,지점명,일시,전운량(10분위)
0,106,동해,2018-10-01 01:00,1
1,106,동해,2018-10-01 02:00,1
2,106,동해,2018-10-01 03:00,0
3,106,동해,2018-10-01 04:00,1
4,106,동해,2018-10-01 05:00,0


In [73]:
cloud.columns

Index(['지점', '지점명', '일시', '전운량(10분위)'], dtype='object')

In [74]:
# 필요한 column 전운량, 날짜 다시 저장 후 column명 변경  
cloud = cloud[['일시', '전운량(10분위)']]
cloud.columns = ['time_date', 'cloud']
cloud.head()

,time_date,cloud
0,2018-10-01 01:00,1
1,2018-10-01 02:00,1
2,2018-10-01 03:00,0
3,2018-10-01 04:00,1
4,2018-10-01 05:00,0


In [75]:
# datetime type 칼럼 추가, 날짜 filtering 하기 위함
cloud['datetime_time_date'] = pd.to_datetime(cloud.time_date)

# str칼럼 추가
cloud['datetime_time_date_str']= cloud['datetime_time_date'].astype(str)

In [76]:
cloud.head()

,time_date,cloud,datetime_time_date,datetime_time_date_str
0,2018-10-01 01:00,1,2018-10-01 01:00:00,2018-10-01 01:00:00
1,2018-10-01 02:00,1,2018-10-01 02:00:00,2018-10-01 02:00:00
2,2018-10-01 03:00,0,2018-10-01 03:00:00,2018-10-01 03:00:00
3,2018-10-01 04:00,1,2018-10-01 04:00:00,2018-10-01 04:00:00
4,2018-10-01 05:00,0,2018-10-01 05:00:00,2018-10-01 05:00:00


In [77]:
# 2018.10.15 이후 데이터만 필터링 , 결측값이 2018 - 10 - 15 이후 이므로 
cloud=cloud[cloud['datetime_time_date'] >= '2018-10-15']
cloud.head()

,time_date,cloud,datetime_time_date,datetime_time_date_str
335,2018-10-15 00:00,9,2018-10-15 00:00:00,2018-10-15 00:00:00
336,2018-10-15 01:00,9,2018-10-15 01:00:00,2018-10-15 01:00:00
337,2018-10-15 02:00,7,2018-10-15 02:00:00,2018-10-15 02:00:00
338,2018-10-15 03:00,6,2018-10-15 03:00:00,2018-10-15 03:00:00
339,2018-10-15 04:00,4,2018-10-15 04:00:00,2018-10-15 04:00:00


In [78]:
cloud.reset_index(drop=True, inplace=True)

In [79]:
# 전운량에서 하늘상태로 맵핑함수 만들기
# 0~2 : 1
# 3~5 : 2
# 6~8 : 3 
# 9~10 : 4
def change_data(x):
    x = int(x)
    if x <= 2 : return 1
    elif (2< x <= 5) :return 2
    elif (5< x <= 8) :return 3
    else : return 4

In [80]:
cloud.cloud.isnull().sum()

0

In [81]:
# 맵핑함수 적용
cloud['cloud'] = cloud['cloud'].apply(change_data)

In [82]:
cloud.head()

,time_date,cloud,datetime_time_date,datetime_time_date_str
0,2018-10-15 00:00,4,2018-10-15 00:00:00,2018-10-15 00:00:00
1,2018-10-15 01:00,4,2018-10-15 01:00:00,2018-10-15 01:00:00
2,2018-10-15 02:00,3,2018-10-15 02:00:00,2018-10-15 02:00:00
3,2018-10-15 03:00,3,2018-10-15 03:00:00,2018-10-15 03:00:00
4,2018-10-15 04:00,2,2018-10-15 04:00:00,2018-10-15 04:00:00


In [83]:
# 시간 칼럼 만들기 
hours = []
for i in range(len(cloud.time_date)):
    hour = cloud['datetime_time_date_str'][i][11:13]
    hours.append(hour)
cloud['hour'] = hours

In [84]:
cloud.head()

,time_date,cloud,datetime_time_date,datetime_time_date_str,hour
0,2018-10-15 00:00,4,2018-10-15 00:00:00,2018-10-15 00:00:00,00
1,2018-10-15 01:00,4,2018-10-15 01:00:00,2018-10-15 01:00:00,01
2,2018-10-15 02:00,3,2018-10-15 02:00:00,2018-10-15 02:00:00,02
3,2018-10-15 03:00,3,2018-10-15 03:00:00,2018-10-15 03:00:00,03
4,2018-10-15 04:00,2,2018-10-15 04:00:00,2018-10-15 04:00:00,04


In [85]:
# 0 없애려고 수행
cloud.hour = cloud.hour.astype(int)
cloud.hour = cloud.hour.astype(str)

In [86]:
cloud.head()

,time_date,cloud,datetime_time_date,datetime_time_date_str,hour
0,2018-10-15 00:00,4,2018-10-15 00:00:00,2018-10-15 00:00:00,0
1,2018-10-15 01:00,4,2018-10-15 01:00:00,2018-10-15 01:00:00,1
2,2018-10-15 02:00,3,2018-10-15 02:00:00,2018-10-15 02:00:00,2
3,2018-10-15 03:00,3,2018-10-15 03:00:00,2018-10-15 03:00:00,3
4,2018-10-15 04:00,2,2018-10-15 04:00:00,2018-10-15 04:00:00,4


In [87]:
# 연 - 월 - 일 만 따로 분리 
cloud['date'] = 0
for i in range(len(cloud.time_date)):
    cloud['date'][i] = cloud['datetime_time_date_str'][i].split(' ')[0]

<ipython-input-87-57906d4a45c7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cloud['date'][i] = cloud['datetime_time_date_str'][i].split(' ')[0]
/Users/jin/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [88]:
cloud.head()

,time_date,cloud,datetime_time_date,datetime_time_date_str,hour,date
0,2018-10-15 00:00,4,2018-10-15 00:00:00,2018-10-15 00:00:00,0,2018-10-15
1,2018-10-15 01:00,4,2018-10-15 01:00:00,2018-10-15 01:00:00,1,2018-10-15
2,2018-10-15 02:00,3,2018-10-15 02:00:00,2018-10-15 02:00:00,2,2018-10-15
3,2018-10-15 03:00,3,2018-10-15 03:00:00,2018-10-15 03:00:00,3,2018-10-15
4,2018-10-15 04:00,2,2018-10-15 04:00:00,2018-10-15 04:00:00,4,2018-10-15


In [89]:
# 키값 만들기
cloud['date_time'] = cloud['date'] + '-' + cloud['hour']
cloud.head()

,time_date,cloud,datetime_time_date,datetime_time_date_str,hour,date,date_time
0,2018-10-15 00:00,4,2018-10-15 00:00:00,2018-10-15 00:00:00,0,2018-10-15,2018-10-15-0
1,2018-10-15 01:00,4,2018-10-15 01:00:00,2018-10-15 01:00:00,1,2018-10-15,2018-10-15-1
2,2018-10-15 02:00,3,2018-10-15 02:00:00,2018-10-15 02:00:00,2,2018-10-15,2018-10-15-2
3,2018-10-15 03:00,3,2018-10-15 03:00:00,2018-10-15 03:00:00,3,2018-10-15,2018-10-15-3
4,2018-10-15 04:00,2,2018-10-15 04:00:00,2018-10-15 04:00:00,4,2018-10-15,2018-10-15-4


In [90]:
# 형태 맞추기
cloud = cloud[['date','hour','cloud','date_time']]

In [91]:
# 해 없는 시간 제거하기 
cloud.hour = cloud.hour.astype(int)
result = []
rej_time = [0,1,2,3,4,5, 21,22,23,24]
for j in rej_time:
    one_indice = []
    for i in range(len(cloud[cloud['hour'] ==j].index)):
        one = cloud[cloud['hour'] ==j].index[i]
        one_indice.append(one)
        
    result.extend(one_indice)    

result

cloud.drop(result, inplace=True)
cloud.reset_index(drop=True, inplace=True)

cloud.head()

,date,hour,cloud,date_time
0,2018-10-15,6,1,2018-10-15-6
1,2018-10-15,7,1,2018-10-15-7
2,2018-10-15,8,1,2018-10-15-8
3,2018-10-15,9,1,2018-10-15-9
4,2018-10-15,10,1,2018-10-15-10


In [92]:
cloud

,date,hour,cloud,date_time
0,2018-10-15,6,1,2018-10-15-6
1,2018-10-15,7,1,2018-10-15-7
2,2018-10-15,8,1,2018-10-15-8
3,2018-10-15,9,1,2018-10-15-9
4,2018-10-15,10,1,2018-10-15-10
...,...,...,...,...
1158,2018-12-31,16,3,2018-12-31-16
1159,2018-12-31,17,2,2018-12-31-17
1160,2018-12-31,18,2,2018-12-31-18
1161,2018-12-31,19,3,2018-12-31-19


# 문제발생 
- 1170개가 되어야 하는데, 7개가 없음...
- 그냥 찾아서 엑셀에서 집어 넣기로 결정

In [93]:
# 시간이 78개가 아닌시간 체크
for i in range(6,21):
    print(cloud[cloud['hour']==i].hour.value_counts())

6    78
Name: hour, dtype: int64
7    78
Name: hour, dtype: int64
8    78
Name: hour, dtype: int64
9    77
Name: hour, dtype: int64
10    78
Name: hour, dtype: int64
11    77
Name: hour, dtype: int64
12    77
Name: hour, dtype: int64
13    77
Name: hour, dtype: int64
14    78
Name: hour, dtype: int64
15    78
Name: hour, dtype: int64
16    78
Name: hour, dtype: int64
17    78
Name: hour, dtype: int64
18    76
Name: hour, dtype: int64
19    77
Name: hour, dtype: int64
20    78
Name: hour, dtype: int64


## 9, 11, 12, 13, 18(2), 19 빔 

In [94]:
cloud[cloud['hour']==9] # 12월 4일 

,date,hour,cloud,date_time
3,2018-10-15,9,1,2018-10-15-9
18,2018-10-16,9,3,2018-10-16-9
33,2018-10-17,9,4,2018-10-17-9
48,2018-10-18,9,4,2018-10-18-9
63,2018-10-19,9,3,2018-10-19-9
78,2018-10-20,9,1,2018-10-20-9
93,2018-10-21,9,1,2018-10-21-9
108,2018-10-22,9,3,2018-10-22-9
123,2018-10-23,9,3,2018-10-23-9
138,2018-10-24,9,1,2018-10-24-9


In [537]:
cloud[cloud['hour']==11] # 12월 4일

,date,hour,cloud,date_time
5,2018-10-15,11,1,2018-10-15-11
20,2018-10-16,11,4,2018-10-16-11
35,2018-10-17,11,4,2018-10-17-11
50,2018-10-18,11,4,2018-10-18-11
65,2018-10-19,11,3,2018-10-19-11
80,2018-10-20,11,1,2018-10-20-11
95,2018-10-21,11,1,2018-10-21-11
110,2018-10-22,11,2,2018-10-22-11
125,2018-10-23,11,4,2018-10-23-11
140,2018-10-24,11,1,2018-10-24-11


In [538]:
cloud[cloud['hour']==12] # 11월 20일

,date,hour,cloud,date_time
6,2018-10-15,12,1,2018-10-15-12
21,2018-10-16,12,3,2018-10-16-12
36,2018-10-17,12,4,2018-10-17-12
51,2018-10-18,12,4,2018-10-18-12
66,2018-10-19,12,3,2018-10-19-12
81,2018-10-20,12,1,2018-10-20-12
96,2018-10-21,12,1,2018-10-21-12
111,2018-10-22,12,2,2018-10-22-12
126,2018-10-23,12,4,2018-10-23-12
141,2018-10-24,12,1,2018-10-24-12


In [539]:
cloud[cloud['hour']==13] # 11월 21일

,date,hour,cloud,date_time
7,2018-10-15,13,1,2018-10-15-13
22,2018-10-16,13,3,2018-10-16-13
37,2018-10-17,13,4,2018-10-17-13
52,2018-10-18,13,4,2018-10-18-13
67,2018-10-19,13,2,2018-10-19-13
82,2018-10-20,13,1,2018-10-20-13
97,2018-10-21,13,1,2018-10-21-13
112,2018-10-22,13,3,2018-10-22-13
127,2018-10-23,13,4,2018-10-23-13
142,2018-10-24,13,1,2018-10-24-13


In [540]:
cloud[cloud['hour']==18] # 10월 30일, 11월 19일

,date,hour,cloud,date_time
12,2018-10-15,18,1,2018-10-15-18
27,2018-10-16,18,3,2018-10-16-18
42,2018-10-17,18,3,2018-10-17-18
57,2018-10-18,18,4,2018-10-18-18
72,2018-10-19,18,1,2018-10-19-18
87,2018-10-20,18,1,2018-10-20-18
102,2018-10-21,18,2,2018-10-21-18
117,2018-10-22,18,4,2018-10-22-18
132,2018-10-23,18,1,2018-10-23-18
147,2018-10-24,18,1,2018-10-24-18


In [541]:
cloud[cloud['hour']==19] # 11월 19일

,date,hour,cloud,date_time
13,2018-10-15,19,1,2018-10-15-19
28,2018-10-16,19,3,2018-10-16-19
43,2018-10-17,19,4,2018-10-17-19
58,2018-10-18,19,3,2018-10-18-19
73,2018-10-19,19,1,2018-10-19-19
88,2018-10-20,19,1,2018-10-20-19
103,2018-10-21,19,1,2018-10-21-19
118,2018-10-22,19,3,2018-10-22-19
133,2018-10-23,19,1,2018-10-23-19
148,2018-10-24,19,1,2018-10-24-19


In [542]:
# 다운받고 엑셀로 채워넣고
# cloud.to_csv('cloud.csv')

In [546]:
# sky_con.csv , cloud.csv 엑셀로 열어서 cloud.csv의 값을 sky_con에 복사
# sky_con.to_csv('sky_con.csv')

## 하늘상태 업데이트 파일 불러옴 

In [95]:
sky_con2 = pd.read_csv('sky_con.csv', index_col=0)

In [96]:
sky_con2['date'] = sky_con['date']

In [97]:
sky_con2['hour'] = sky_con2['hour'].astype(str)

In [100]:
sky_con2[5300:]

,date,hour,sky_con,date_time
5300,2018-12-20,11,3,2018-12-20-11
5301,2018-12-20,12,3,2018-12-20-12
5302,2018-12-20,13,3,2018-12-20-13
5303,2018-12-20,14,2,2018-12-20-14
5304,2018-12-20,15,1,2018-12-20-15
...,...,...,...,...
5470,2018-12-31,16,3,2018-12-31-16
5471,2018-12-31,17,2,2018-12-31-17
5472,2018-12-31,18,2,2018-12-31-18
5473,2018-12-31,19,3,2018-12-31-19


## merge 수행

In [102]:
# df = pd.merge(humidity, rain)
# df = pd.merge(df, raintype)
# df = pd.merge(df, temp)
# df = pd.merge(df, thunder)
# df = pd.merge(df, wind_angle)
# df = pd.merge(df, wind_velo)
# df = pd.merge(df, sky_con)
# df.head()

# 하늘상태 업데이트 후
df = pd.merge(humidity, rain)
df = pd.merge(df, raintype)
df = pd.merge(df, temp)
df = pd.merge(df, thunder)
df = pd.merge(df, wind_angle)
df = pd.merge(df, wind_velo)
df = pd.merge(df, sky_con2)

In [103]:
df.tail()

,date,hour,humidity,date_time,rain,rain_type,temp,thunder,wind_angle,wind_velo,sky_con
5470,2018-12-31,16,26.0,2018-12-31-16,0.0,0.0,-2.7,-1.0,228.0,1.4,3
5471,2018-12-31,17,18.0,2018-12-31-17,0.0,0.0,-1.2,-1.0,253.0,1.8,2
5472,2018-12-31,18,24.0,2018-12-31-18,0.0,0.0,-2.7,-1.0,217.0,1.8,2
5473,2018-12-31,19,22.0,2018-12-31-19,0.0,0.0,-2.7,-1.0,209.0,1.1,3
5474,2018-12-31,20,23.0,2018-12-31-20,0.0,0.0,-2.7,-1.0,217.0,1.2,3


In [104]:
df.columns

Index(['date', 'hour', 'humidity', 'date_time', 'rain', 'rain_type', 'temp',
       'thunder', 'wind_angle', 'wind_velo', 'sky_con'],
      dtype='object')

In [105]:
df=df[['date', 'hour', 'humidity','rain', 'rain_type', 'temp',
       'thunder', 'wind_angle', 'wind_velo', 'sky_con','date_time']]

In [106]:
df.tail()

,date,hour,humidity,rain,rain_type,temp,thunder,wind_angle,wind_velo,sky_con,date_time
5470,2018-12-31,16,26.0,0.0,0.0,-2.7,-1.0,228.0,1.4,3,2018-12-31-16
5471,2018-12-31,17,18.0,0.0,0.0,-1.2,-1.0,253.0,1.8,2,2018-12-31-17
5472,2018-12-31,18,24.0,0.0,0.0,-2.7,-1.0,217.0,1.8,2,2018-12-31-18
5473,2018-12-31,19,22.0,0.0,0.0,-2.7,-1.0,209.0,1.1,3,2018-12-31-19
5474,2018-12-31,20,23.0,0.0,0.0,-2.7,-1.0,217.0,1.2,3,2018-12-31-20


In [107]:
df.isnull().sum()

date          0
hour          0
humidity      0
rain          0
rain_type     0
temp          0
thunder       0
wind_angle    0
wind_velo     0
sky_con       0
date_time     0
dtype: int64

In [108]:
df.tail()

,date,hour,humidity,rain,rain_type,temp,thunder,wind_angle,wind_velo,sky_con,date_time
5470,2018-12-31,16,26.0,0.0,0.0,-2.7,-1.0,228.0,1.4,3,2018-12-31-16
5471,2018-12-31,17,18.0,0.0,0.0,-1.2,-1.0,253.0,1.8,2,2018-12-31-17
5472,2018-12-31,18,24.0,0.0,0.0,-2.7,-1.0,217.0,1.8,2,2018-12-31-18
5473,2018-12-31,19,22.0,0.0,0.0,-2.7,-1.0,209.0,1.1,3,2018-12-31-19
5474,2018-12-31,20,23.0,0.0,0.0,-2.7,-1.0,217.0,1.2,3,2018-12-31-20


In [596]:
# df.to_csv('donghae_weather_2018.csv')